 # Loading an HCA matrix into Bioconductor
 
This vignette illustrates requesting an expression matrix from the HCA matrix service and loading it into a Bioconductor R object.

First, install and import some dependencies:

In [1]:
install.packages("downloader")
install.packages("BiocManager")

library("downloader")
library("BiocManager")

BiocManager::install("LoomExperiment")
library(LoomExperiment)

library("httr")


The downloaded binary packages are in
	/var/folders/nl/dgln31tj7l35g879d6f_tjtc0000gn/T//RtmpDOn1ah/downloaded_packages

The downloaded binary packages are in
	/var/folders/nl/dgln31tj7l35g879d6f_tjtc0000gn/T//RtmpDOn1ah/downloaded_packages


Bioconductor version 3.9 (BiocManager 1.30.4), R 3.6.0 (2019-04-26)
Installing package(s) 'LoomExperiment'



The downloaded binary packages are in
	/var/folders/nl/dgln31tj7l35g879d6f_tjtc0000gn/T//RtmpDOn1ah/downloaded_packages


Update old packages: 'blob', 'bookdown', 'boot', 'digest', 'dplyr',
  'googleAuthR', 'hms', 'IRkernel', 'pillar', 'rmarkdown', 'vctrs'
Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sappl

Now, we're going to make some requests to describe what fields and values we can filter on when we're selecting our matrix.

In [2]:
r <- GET("https://matrix.data.humancellatlas.org/v1/filters")
content(r)

[[1]]
[1] "cell_suspension.provenance.document_id"

[[2]]
[1] "genes_detected"

[[3]]
[1] "specimen_from_organism.provenance.document_id"

[[4]]
[1] "specimen_from_organism.genus_species.ontology"

[[5]]
[1] "specimen_from_organism.genus_species.ontology_label"

[[6]]
[1] "donor_organism.human_specific.ethnicity.ontology"

[[7]]
[1] "donor_organism.human_specific.ethnicity.ontology_label"

[[8]]
[1] "donor_organism.diseases.ontology"

[[9]]
[1] "donor_organism.diseases.ontology_label"

[[10]]
[1] "donor_organism.development_stage.ontology"

[[11]]
[1] "donor_organism.development_stage.ontology_label"

[[12]]
[1] "derived_organ_ontology"

[[13]]
[1] "derived_organ_label"

[[14]]
[1] "derived_organ_part_ontology"

[[15]]
[1] "derived_organ_part_label"

[[16]]
[1] "library_preparation_protocol.provenance.document_id"

[[17]]
[1] "library_preparation_protocol.input_nucleic_acid_molecule.ontology"

[[18]]
[1] "library_preparation_protocol.input_nucleic_acid_molecule.ontology_label"

[[19]]
[1] "library_preparation_protocol.library_construction_method.ontology"

[[20]]
[1] "library_preparation_protocol.library_construction_method.ontology_label"

[[21]]
[1] "library_preparation_protocol.end_bias"

[[22]]
[1] "library_preparation_protocol.strand"

[[23]]
[1] "project.provenance.document_id"

[[24]]
[1] "project.project_core.project_short_name"

[[25]]
[1] "project.project_core.project_title"

[[26]]
[1] "analysis_protocol.provenance.document_id"

[[27]]
[1] "dss_bundle_fqid"

[[28]]
[1] "analysis_protocol.protocol_core.protocol_id"

[[29]]
[1] "analysis_working_group_approval_status"

That's the list of metadata fields we can filter on when requesting the matrix. We can describe any of them with further API calls:

In [3]:
r <- GET("https://matrix.data.humancellatlas.org/v1/filters/project.project_core.project_short_name")
print(content(r))

$cell_counts
$cell_counts$`Fetal/Maternal Interface`
[1] 1

$cell_counts$`Single cell RNAseq characterization of cell types produced over time in an in vitro model of human inhibitory interneuron differentiation.`
[1] 1733

$cell_counts$`Single cell transcriptome analysis of human pancreas`
[1] 2544


$field_description
[1] "A short name for the project."

$field_name
[1] "project.project_core.project_short_name"

$field_type
[1] "categorical"



In [4]:
r <- GET("https://matrix.data.humancellatlas.org/v1/filters/genes_detected")
print(content(r))

$field_description
[1] "Count of genes with a non-zero count."

$field_name
[1] "genes_detected"

$field_type
[1] "numeric"

$maximum
[1] 13108

$minimum
[1] 358



For categorical data, we see the number of cells associated with each category. For numeric, we see the range of value. If we want to request a matrix based on these metadata values, we can add them to the filter in the body of a POST request to the matrix service:

In [5]:
payload = list(
    filter =  list(
          op = "and", 
          value = list(
              list(op = "=", value = "Single cell transcriptome analysis of human pancreas",
                   field = "project.project_core.project_short_name"),
              list(op = ">=", value = 300,
                   field = "genes_detected")
    )),
    format = "loom"
)
r <- POST("https://matrix.data.humancellatlas.org/v1/matrix", body = payload, encode = "json")
response <- content(r)
print(response)

$eta
[1] ""

$matrix_url
[1] ""

$message
[1] "Job started."

$request_id
[1] "ca657a20-5896-4583-80fc-16c4f02657da"

$status
[1] "In Progress"



That call responds right away and tells us that the matrix is being prepared. We can use the request_id to wait until the matrix is done.

In [6]:
request_id <- response["request_id"]
status <- response["status"]
message(status)
while (status != "Complete") 
{
    url = paste("https://matrix.data.humancellatlas.org/v1/matrix/", request_id, sep = "")
    r <- GET(url)
    response <- content(r)
    status = response["status"]
    message(status)
    Sys.sleep(15)
}
print(response)

In Progress
In Progress
In Progress
In Progress
In Progress
In Progress
In Progress
In Progress
In Progress
In Progress
In Progress
Complete


$eta
[1] ""

$matrix_url
[1] "https://s3.amazonaws.com/dcp-matrix-service-results-prod/ca657a20-5896-4583-80fc-16c4f02657da.loom"

$message
[1] "Request ca657a20-5896-4583-80fc-16c4f02657da has successfully completed. The resultant expression matrix is available for download at https://s3.amazonaws.com/dcp-matrix-service-results-prod/ca657a20-5896-4583-80fc-16c4f02657da.loom"

$request_id
[1] "ca657a20-5896-4583-80fc-16c4f02657da"

$status
[1] "Complete"



Now, that the matrix is ready, we can download it. The file we download is a loom-formatted matrix. Loom is the default matrix format, but others can be specified in the matrix request (csv, mtx).

In [7]:
matrix_file_url = unlist(response["matrix_url"])

download.file(url=matrix_file_url,
              destfile='matrix.loom', method='curl')

# HCA Matrix Service Loom Output

The Loom format is documented more fully, along with code samples, [here](https://linnarssonlab.org/loompy/index.html).

Per Loom [conventions](https://linnarssonlab.org/loompy/conventions/index.html), columns in the loom-formatted expression matrix represent cells, and rows represent genes. The column and row attributes follow Loom conventions where applicable as well: `CellID` uniquely identifies a cell, `Gene` is a gene name, and `Accession` is an ensembl gene id.

Descriptions of the remaining metadata fields are available at the [HCA Data Browser](https://prod.data.humancellatlas.org/metadata).

And finally, we can `import` the loom file into a `Bioconductor::SingleCellLoomExperiment` object for further analysis in R.

In [8]:
scle <- import("./matrix.loom", type="SingleCellLoomExperiment")
scle

class: SingleCellLoomExperiment 
dim: 63925 2544 
metadata(3): CreationDate LOOM_SPEC_VERSION last_modified
assays(1): matrix
rownames: NULL
rowData names(7): Accession Gene ... featuretype isgene
colnames: NULL
colData names(30): CellID analysis_protocol.protocol_core.protocol_id
  ... specimen_from_organism.genus_species.ontology_label
  specimen_from_organism.provenance.document_id
reducedDimNames(0):
spikeNames(0):
rowGraphs(0): NULL
colGraphs(0): NULL

The `SingleCellLoomExperiment` also adheres to Loom [conventions](https://linnarssonlab.org/loompy/conventions/index.html) representing features as rows and samples as columns. Expression data is available via the `assays()` method, specifying a named assay.

In [9]:
assays(scle)$matrix

<63925 x 2544> DelayedMatrix object of type "double":
            [,1]    [,2]    [,3] ... [,2543] [,2544]
    [1,]       0       0       0   .       0       0
    [2,]       0       0       0   .       0       0
    [3,]      11       0      37   .       0       6
    [4,]       0       0       0   .     101       0
    [5,]       0       0       0   .       0       0
     ...       .       .       .   .       .       .
[63921,]    0.00    0.00    0.00   .       0       2
[63922,]    0.00    0.00    0.00   .       0       0
[63923,]    0.00    0.00    0.00   .       0       0
[63924,]    0.99    0.00    0.00   .       0       0
[63925,]    0.00    0.00    0.00   .       0       0

Row and column attribute data are available through `rowData()` and `colData()` methods respectively.

In [10]:
rowData(scle)

DataFrame with 63925 rows and 7 columns
            Accession        Gene  chromosome featureend featurestart
          <character> <character> <character>  <integer>    <integer>
1     ENSG00000000003      TSPAN6        chrX  100639991    100627109
2     ENSG00000000005        TNMD        chrX  100599885    100584802
3     ENSG00000000419        DPM1       chr20   50958555     50934867
4     ENSG00000000457       SCYL3        chr1  169894267    169849631
5     ENSG00000000460    C1orf112        chr1  169854080    169662007
...               ...         ...         ...        ...          ...
63921 ENSG00000284744  AL591163.1        chr1    6770038      6767954
63922 ENSG00000284745  AL589702.1        chr1    2968707      2960658
63923 ENSG00000284746 AC068587.10        chr8   12601376     12601158
63924 ENSG00000284747  AL034417.4        chr1    8005312      7991134
63925 ENSG00000284748  AL513220.1        chr1   37607336     37596126
                             featuretype    isgene

For more examples on usage, see the Bioconductor documentation for the following parent classes of `SingleCellLoomExperiment`:

- [SummarizedExperiment](http://bioconductor.org/packages/release/bioc/vignettes/SummarizedExperiment/inst/doc/SummarizedExperiment.html#anatomy-of-a-summarizedexperiment)
- [SingleCellExperiment](https://bioconductor.org/packages/devel/bioc/vignettes/scater/inst/doc/vignette-intro.html#3_calculating_a_variety_of_expression_values)
- [LoomExperiment](http://bioconductor.org/packages/release/bioc/vignettes/LoomExperiment/inst/doc/LoomExperiment.html)